In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects

# Load the data
try:
    df = pd.read_csv('NIR_in_vitro.csv')
    if len(df.columns) == 1:
        df = pd.read_csv('NIR_in_vitro.csv', delimiter='\t')
except FileNotFoundError:
    print("Error: 'NIR_in_vitro.csv' not found. Ensure the CSV is in the script directory.")
    exit()

# New list of concentration prefixes
concentrations_prefixes = ['Control', 'C50', 'C100', 'C200', 'C400']
mean_cols = []
sem_cols = []

# Process each group
for prefix in concentrations_prefixes:
    if prefix == 'Control':
        current_temp_cols = [col for col in df.columns if col.startswith('Control')]
    else:
        current_temp_cols = [col for col in df.columns if col.startswith(prefix + '_') or col.startswith(prefix + '.') or col.startswith(prefix)]

    if current_temp_cols:
        mean_col_name = f'Mean_{prefix}'
        sem_col_name = f'SEM_{prefix}'
        mean_cols.append(mean_col_name)
        sem_cols.append(sem_col_name)
        df[mean_col_name] = df[current_temp_cols].mean(axis=1)
        df[sem_col_name] = df[current_temp_cols].sem(axis=1)
    else:
        print(f"Warning: No columns found for prefix '{prefix}'. Skipping group.")

# Plotting
plt.figure(figsize=(7, 5))

# Define colors, markers, and labels for each group
plot_colors = ['peru', 'teal', 'green', 'indigo', 'crimson']
plot_markers = ['o', 's', '^', 'v', 'D']
plot_labels = ['Control', '50 µg/ml', '100 µg/ml', '200 µg/ml', '400 µg/ml']

for i, mean_col in enumerate(mean_cols):
    sem_col = sem_cols[i]
    color = plot_colors[i]
    marker = plot_markers[i]
    label = plot_labels[i]

    errorbar = plt.errorbar(df['Time(s)'].values, df[mean_col], yerr=df[sem_col],
                            color=color, linewidth=2, capsize=4, fmt='none')
    plt.plot(df['Time(s)'].values, df[mean_col], color=color, linewidth=2, linestyle='-',
             marker=marker, markersize=6, label=label)

    # Path effects for caps
    for cap in errorbar[1]:
        cap.set_markeredgewidth(1)
        cap.set_path_effects([path_effects.Stroke(linewidth=1), path_effects.Normal()])

# Font properties
font_properties = {'fontweight': 'bold', 'fontsize': 18, 'fontfamily': 'Arial'}
number_font_properties = {'fontweight': 'bold', 'fontsize': 18, 'fontfamily': 'Arial'}
legend_font_properties = {'size': 14, 'family': 'Arial'}

plt.xlabel('Time (s)', **font_properties)
plt.ylabel('Temperature (°C)', **font_properties)

plt.xticks(np.arange(0, df['Time(s)'].max() + 1, 100), ha='center', **number_font_properties)
plt.yticks(np.arange(20, 70, 5), **number_font_properties)

plt.xlim(left=0)
plt.ylim(bottom=20)

plt.tick_params(axis='both', which='both', length=10, width=2, labelsize=18)

# Border settings
ax = plt.gca()
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Legend
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), prop=legend_font_properties, frameon=False, labelspacing=1)

# Save figure
plt.savefig('NIR_in_vitro_graph.tif', dpi=600, format='tif', bbox_inches='tight')
plt.show()